In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from signals import *
from frequencyestimator import *
import time
import copy

sns.set_style("whitegrid")
sns.despine(left=True, bottom=True)
sns.set_context("poster", font_scale = .45, rc={"grid.linewidth": 0.8})

<Figure size 640x480 with 0 Axes>

# Example Implementation

Here we provide a minimal working example demonstrating how to use the code to estimate the amplitude using the ESPIRIT algorithm.

In [4]:
# For reproducibility
np.random.seed(8)
# Set the per oracle noise parameter (See Eq. 18)
# eta=1e-4
eta=0
# Set the array parameters (See Thm. II.2 and Eq. 12) 
narray = [3, 3, 3, 3, 2, 2, 2, 2]
narray = [2]*8
# Set the actual amplitude
a=0.1
theta = np.arcsin(a)

# This sets up the simulation that simulates the measured amplitudes at the various physical locations.
# It uses a C=1.5 value, which corresponds to the sampling schedule given in Eq. 16. The variable C here 
# is the parameter K in the paper.
ula_signal = TwoqULASignal(M=narray, C=50)
# Number of Monte Carlo trials used to estimate statistics. We tend to use 500 in the paper. Choose 100 here for speed.
num_mc = 10
thetas = np.zeros(num_mc, dtype = float)
errors = np.zeros(num_mc, dtype = float)

# Sets up the ESPIRIT object to estimate the amplitude
espirit = ESPIRIT()

for k in range(num_mc):
    # ula_signal.n_samples = [500]*len(ula_signal.n_samples)
    signal = ula_signal.estimate_signal(ula_signal.n_samples, theta, eta=eta)
    # This estimates the covariance matrix of Eq. 8 using the approch given in DOI:10.1109/LSP.2015.2409153
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    # This estimates the angle using the ESPIRIT algorithm
    theta_est, _ = espirit.estimate_theta_toeplitz(R)
    # Estimate the error between estimated a and actual a
    error = np.abs(np.sin(theta)-np.sin(theta_est)) 
    thetas[k] = theta_est            
    errors[k] = error

# Compute the total number of queries. The additional count of ula_signal.n_samples[0] is to 
# account for the fact that the Grover oracle has two invocations of the unitary U, but is 
# preceded by a single invocation of U (see Eq. 2 in paper). This accounts for the shots required
# for that single U operator, which costs half as much as the Grover oracle.
num_queries = np.sum(np.array(ula_signal.depths)*np.array(ula_signal.n_samples)) + ula_signal.n_samples[0]/2
# Compute the maximum single query
max_single_query = np.max(ula_signal.depths)

print(f'Array parameters: {narray}')
print(f'Depths: {ula_signal.depths}')
print(f'Samples: {ula_signal.n_samples}')
print(f'Number of queries: {num_queries}')
print(f'theta: {theta/np.pi}')
print(f'Ave theta estimated: {np.mean(thetas)/np.pi}')
print(f'a = {a}; a_est = {np.sin(np.mean(thetas))}')
print(f'Max Single Query: {max_single_query}')
print(f'99% percentile: {np.percentile(errors, 99):e}')
print(f'95% percentile: {np.percentile(errors, 95):e}')
print(f'68% percentile: {np.percentile(errors, 68):e}')
print(f'99% percentile constant: {np.percentile(errors, 99)*num_queries:f}')
print(f'95% percentile constant: {np.percentile(errors, 95)*num_queries:f}')
print(f'68% percentile constant: {np.percentile(errors, 68)*num_queries:f}')
print(f'99% percentile max constant: {np.percentile(errors, 99)*max_single_query:f}')
print(f'95% percentile max constant: {np.percentile(errors, 95)*max_single_query:f}')
print(f'68% percentile max constant: {np.percentile(errors, 68)*max_single_query:f}')
print()


Array parameters: [2, 2, 2, 2, 2, 2, 2, 2]
Depths: [  0   1   2   4   8  16  32  64 128]
Samples: [450, 425, 400, 375, 350, 325, 300, 275, 250]
Number of queries: 70150.0
theta: 0.03188428042925993
Ave theta estimated: 0.031898479438149635
a = 0.1; a_est = 0.10004438380460062
Max Single Query: 128
99% percentile: 5.079959e-04
95% percentile: 5.037887e-04
68% percentile: 3.964951e-04
99% percentile constant: 35.635911
95% percentile constant: 35.340779
68% percentile constant: 27.814130
99% percentile max constant: 0.065023
95% percentile max constant: 0.064485
68% percentile max constant: 0.050751

